In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [ ]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

# Get sample data

In [ ]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=1, target_snr=25)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

# Regularization Optimizations

In [ ]:
from invert.evaluate import corr, nmse
from invert.solvers.lucas import SolverLUCAS

solver = SolverLUCAS()
solver.make_inverse_operator(fwd, evoked, verbose=0)
solver.optimize_weights(fwd, info)
solver.plot_weights()
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

# Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))

In [ ]:
from invert import Solver
from invert.evaluate import corr, nmse
from invert.adapters import contextualize_bd, focuss

solver_ = Solver(solver="laura")

solver_.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver_.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))
print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

# # Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver_.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver_.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))


# Adapt

In [ ]:
from invert.adapters import contextualize_bd, contextualize

stc_hat_cbd = contextualize_bd(stc_hat, fwd, lstm_look_back=10, num_units=128, num_epochs=50, verbose=0)
stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

# Test/ Evaluation

In [30]:
solver_name = "LUCAS"
print(solver_name)
solver = Solver(solver=solver_name)
if (not solver_name in solvers) or ("sparse" in solver_name.lower() or "bayes" in solver_name.lower()):
    solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
# stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
error = np.mean(corr(stc.data, stc_hat.data))
errors[solver_name].append( error )

LUCAS
Preparing MNE
Preparing wMNE
Preparing dSPM
alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
Preparing LORETA
Preparing sLORETA
Preparing eLORETA
Preparing LAURA
Preparing S-MAP
Preparing Fully-Connected
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 5000/5000 [00:00<00:00, 23475.20it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 5000/5000 [00:04<00:00, 1129.43it/s]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average referenc

preprocess data
fit model


In [25]:
from invert import Solver
from invert.config import all_solvers
from invert.evaluate import nmse, corr
settings = dict(number_of_sources=(1,10), extents=(1, 40), duration_of_trial=1, target_snr=(1,25))


errors = {sname: [] for sname in all_solvers}
solvers = dict()
for i in range(1):
    # print(i)
    sim = Simulation(fwd, info, settings).simulate(2)
    stc = sim.source_data[0]
    evoked = sim.eeg_data[0].average()

    for solver_name in all_solvers:
        print(solver_name)
        solver = Solver(solver=solver_name)
        if (not solver_name in solvers) or ("sparse" in solver_name.lower() or "bayes" in solver_name.lower()):
            solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
        stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        error = np.mean(corr(stc.data, stc_hat.data))
        errors[solver_name].append( error )

import pickle as pkl
fn = "errors.pkl"
with open(fn, 'wb') as f:
    pkl.dump(errors, f)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 333.44it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 10.31it/s]


MNE
wMNE
dSPM
alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
LORETA
sLORETA
eLORETA
LAURA
Backus-Gilbert
S-MAP
Multiple Sparse Priors
Bayesian LORETA
Bayesian MNE
Bayesian Beamformer
Bayesian Beamformer LORETA
Fully-Connected
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


...

100%|██████████| 5000/5000 [00:04<00:00, 1090.71it/s]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = [epoch.set_eeg_reference('average', projection=True, verbose=0) for epoch in epochs]
c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\esinet\util\util.py:122: RuntimeWarning: An average referenc

preprocess data
fit model
1/1 [==============================] - 0s 47ms/step


c:\Users\lukas\virtualenvs\invertenv\lib\site-packages\scipy\optimize\_minimize.py:863: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


LUCAS


AttributeError: LUCAS is not available. Please choose from one of the following: ['MNE', 'wMNE', 'dSPM', 'LORETA', 'sLORETA', 'eLORETA', 'LAURA', 'Backus-Gilbert', 'S-MAP', 'Multiple Sparse Priors', 'Bayesian LORETA', 'Bayesian MNE', 'Bayesian Beamformer', 'Bayesian Beamformer LORETA', 'Fully-Connected', 'LUCAS']

In [ ]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
df = pd.DataFrame(errors)
sorted_index = df.median().sort_values().index
df = df[sorted_index]

plt.figure()
sns.boxplot(data=df)
plt.title("Correlation with ground truth")


df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
df_mean_var["Method"] = df_mean_var.index
display(df_mean_var)

plt.figure()
sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
plt.xlabel("Median")
plt.ylabel("Variance")


In [ ]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)